In [3]:
pip show langchain_openai

Name: langchain-openai
Version: 0.1.23
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get("OPENAI_API_KEY")

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke("Explain Quantum Mechanics in one sentence", model = "gpt-4o-mini")
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit both wave-like and particle-like properties, and their states are fundamentally probabilistic rather than deterministic.


In [5]:
from langchain.schema import(
    SystemMessage, 
    AIMessage, 
    HumanMessage
)
messages = [
    SystemMessage(content = "You are a physicist and respond only in German"),
    HumanMessage(content="Explain quantum mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch probabilistische Wellenfunktionen.


## Caching LLM Responses
- Using Langchain's `InMemoryCache()`
- Using `SQLiteCache()`

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

In [7]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Explain quantum mechanics in 5 words"
output = llm.invoke(prompt)

CPU times: total: 31.2 ms
Wall time: 1.32 s


In [8]:
print(output)



Wave-like behavior of subatomic particles.


In [9]:
%%time
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 0 ns


'\n\nWave-like behavior of subatomic particles.'

In [10]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path = ".langchain.db"))

In [11]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 717 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "

In [12]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: total: 344 ms
Wall time: 361 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "

## LLM Streaming

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")
prompt = "Write a rock song about the moon and a raven (2 paras)"
print(llm.invoke(prompt).content)

**Title: "Raven's Moonlight"**

(Verse 1)  
Underneath the silver glow, where shadows start to creep,  
A raven calls from the edge of night, secrets it will keep.  
With wings like midnight feathers, it dances on the breeze,  
Whispering to the moonlit sky, a song that brings me to my knees.  
The stars like diamonds flicker, in a canvas dark and deep,  
As the raven soars through the night, while the world around me sleeps.  

(Chorus)  
Oh, raven of the moonlight, take me where you fly,  
Through the mysteries of the night, beneath the endless sky.  
We'll chase the dreams that haunt us, with every haunting cry,  
In the shadows of the moonlight, together we will rise.  
With the fire of the night, and the howl of the wind,  
We'll carve our names in starlight, let the wild ride begin!  


## The main difference between `ChatOpenAI` and `OpenAI` in the langchain_openai library lies in their ***usage and functionality***. `ChatOpenAI` is specifically *designed for chat models*, where the input and output are ***conversational messages***. On the other hand, `OpenAI` is used for *interfacing with language models* (LMS) and is ***NOT specifically tailored for chat interactions***.

In [15]:
for i in llm.stream(prompt):
    print(i.content, end = "", flush=True)

**Verse 1**  
In the shadow of the night, where the silver light glows,  
A raven takes to flight, with secrets only it knows.  
Wings like whispers, cutting through the dark,  
It dances with the moon, leaving behind its mark.  
Howling winds, they call, while the stars ignite the sky,  
Together they weave tales of love and lullabies.  

**Chorus**  
Oh, moonlight dreams and midnight schemes,  
With a raven soaring high, breaking all the seams.  
In a world of shadows, where the wild hearts roam,  
We’ll chase the night together, never far from home.  

## PromptTemplate

In [18]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}'''

prompt_template = PromptTemplate.from_template(template = template)

prompt = prompt_template.format(virus = "Monkey Pox", language = "English")
prompt

'You are an experienced virologist.\nWrite a few sentences about the following virus "Monkey Pox" in English'

In [19]:
llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)
output = llm.invoke(prompt).content

for i in llm.stream(prompt):
    print(i.content, end = "", flush=True)

Monkeypox is a viral zoonotic disease caused by the monkeypox virus, which is part of the Orthopoxvirus genus, the same family that includes smallpox. Initially identified in laboratory monkeys in 1958, the virus primarily circulates in certain animal populations in Central and West Africa, but human cases have been reported outside these regions. Transmission to humans can occur through direct contact with infected animals, bodily fluids, or contaminated materials, and it can also spread between humans through respiratory droplets or skin lesions. Symptoms typically include fever, rash, and swollen lymph nodes, and while the disease is generally self-limiting, it can lead to severe complications in some cases. Vaccination against smallpox has been shown to provide some cross-protection against monkeypox.

## ChatPromptTemplates

In [20]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [22]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "country": "China",
            "population": "1,439,323,776"
        },
        {
            "rank": 2,
            "country": "India",
            "population": "1,380,004,385"
        },
        {
            "rank": 3,
            "country": "United States",
            "population": "331,002,651"
        },
        {
            "rank": 4,
            "country": "Indonesia",
            "population": "273,523,615"
        },
        {
            "rank": 5,
            "country": "Pakistan",
            "population": "220,892,340"
        }
    ]
}
